In [14]:
import numpy as np
import pandas as pd
import os
import glob
import shutil
import tqdm
import torch
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from dataset_tools import OwnDataset, MNIST
import matplotlib.pyplot as plt

from plotting import show_img


In [15]:
data_path = r"/Users/rasmusjensen/Documents/02466-Cell-Profiling/data_subset"

folder_paths = glob.glob(os.path.join(data_path, "*"))

print(len(folder_paths))

# folder_paths

subset_folder_paths = folder_paths[:50]


2


In [16]:
# for folder_path in subset_folder_paths:
#     folder_name = folder_path.split("\\")[-1]
#     new_folder_path = os.path.join(new_data_path, folder_name)
#     shutil.copytree(folder_path, new_folder_path)
#     print(f"Copying {folder_name} to {new_folder_path}")

In [17]:
# load metadata
df_metadata = pd.read_parquet("metadata.parquet")


# npy_paths = glob.glob(os.path.join(data_path, "*", "*.npy"))
npy_paths = glob.glob(os.path.join(data_path, "*", "*.npy"))
len(npy_paths)



7774

In [18]:
# use tqdm to see progress
save_path = "data_subset\\"

# get the moa for each file, and put it in a folder named after the moa
for i, npy_path in enumerate(tqdm.tqdm(npy_paths)):
    data = np.load(npy_path)
    # data = (data / np.max(data) * 255).astype(np.int8)
    lookup_path = npy_path.split("\\")[-1]
    moa = df_metadata[df_metadata["Single_Cell_Image_Name"] == lookup_path]["moa"].values[0]

    # get file under data_folder named moa
    # if not exists, create it
    os.makedirs(os.path.join(save_path, moa), exist_ok=True)
    # save the file
    file_path = os.path.join(save_path, moa, lookup_path)
    np.save(file_path, data)

    
    # if i == 0:
    #     total_data = np.expand_dims(data, axis=0)
    # else:
    #     total_data = np.vstack((total_data, np.expand_dims(data, axis=0)))
    # if i == 1000:
    #     break

  0%|          | 0/7774 [00:00<?, ?it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [25]:

# Initialize lists to store extracted well identifiers and their associated values
well_ids = []
data_values = []

# Loop through the data to extract well identifiers and values
for entry in data:
    parts = entry.split('_')
    well_id = parts[0]
    data_value = int(parts[-1])  # Assuming the data value is the last part of the name and is an integer
    well_ids.append(well_id)
    data_values.append(data_value)

# Create a DataFrame from the extracted data
df = pd.DataFrame({'well_id': well_ids, 'data_value': data_values})

# Group by well_id and compute the mean
mean_values = df.groupby('well_id')['data_value'].mean().reset_index()

import ace_tools as tools; tools.display_dataframe_to_user(name="Mean Values by Well", dataframe=mean_values)

# Display the result
print(mean_values)


AttributeError: 'numpy.ndarray' object has no attribute 'split'

In [ ]:
dataset = MNIST(train=True, transform=transforms.ToTensor())
dataset

In [ ]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
x, y = next(iter(loader))

y

In [ ]:

def npy_loader(path):
    """
    load a npy file and change the dtype to int8
    """
    sample = np.load(path)

    sample = (sample / np.max(sample) * 255).astype(np.uint8)

    sample = torch.from_numpy(sample)
    # make channel the first dimension
    sample = sample.permute(2, 0, 1)

    return sample


In [ ]:
dataset = datasets.DatasetFolder("data_subset/singh_cp_pipeline_singlecell_images", loader=npy_loader, extensions=('.npy',))
# dataset = OwnDataset()

dataset[3050][0].shape
show_img(dataset[3051][0])

In [ ]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# metadata

In [ ]:
# df_metadata = pd.read_csv("metadata.csv", index_col=0)
# df_features = pd.read_csv("CPfeatures.csv", index_col=0) # very large file (4.2 GB)

# read from parquet instead of csv

# df_features = pd.read_parquet("CPfeatures.parquet")

In [ ]:
df_metadata.head()
# df_features.head()

In [ ]:
df_metadata["Multi_Cell_Image_Id"].unique()

In [48]:
npy_paths[-4]

'/Users/rasmusjensen/Documents/02466-Cell-Profiling/data_subset/Protein synthesis/B03_s3_w1405B7123-180B-47BF-B521-42AB6A2FB8F0_11.npy'

In [45]:
import re

file_path = '/Users/rasmusjensen/Documents/02466-Cell-Profiling/data_subset/DMSO/B02_s2_w1E728D3CA-2152-49A6-AD0E-B24D6089C18B_193.npy'

# Use regular expressions to extract the value
for name in npy_paths:
    match = re.search(r'/([^/]+)_s\d+_w\d', file_path)
    if match:
        extracted_value = match.group(1)
        print(extracted_value)  # Output: B02
    else:
        print("Pattern not found")

B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02
B02


In [49]:
for file_path in npy_paths:
    parts = file_path.split('/')
    print(parts)
    last_part = parts[-1]
    sub_parts = last_part.split('_')
    extracted_value = sub_parts[0]
    print(extracted_value) 

['', 'Users', 'rasmusjensen', 'Documents', '02466-Cell-Profiling', 'data_subset', 'DMSO', 'B02_s4_w1DD020500-6669-4C15-B182-C905E185976C_16.npy']
B02
['', 'Users', 'rasmusjensen', 'Documents', '02466-Cell-Profiling', 'data_subset', 'DMSO', 'B02_s3_w1AE61D0E7-AF34-42E7-89FE-7D22560989F4_52.npy']
B02
['', 'Users', 'rasmusjensen', 'Documents', '02466-Cell-Profiling', 'data_subset', 'DMSO', 'B02_s1_w1B1A7ADEA-8896-4C7D-8C63-663265374B72_32.npy']
B02
['', 'Users', 'rasmusjensen', 'Documents', '02466-Cell-Profiling', 'data_subset', 'DMSO', 'B02_s1_w1B1A7ADEA-8896-4C7D-8C63-663265374B72_26.npy']
B02
['', 'Users', 'rasmusjensen', 'Documents', '02466-Cell-Profiling', 'data_subset', 'DMSO', 'B02_s2_w156A83C2E-286E-4F75-A543-66F74D9B871C_15.npy']
B02
['', 'Users', 'rasmusjensen', 'Documents', '02466-Cell-Profiling', 'data_subset', 'DMSO', 'B02_s3_w1AE61D0E7-AF34-42E7-89FE-7D22560989F4_46.npy']
B02
['', 'Users', 'rasmusjensen', 'Documents', '02466-Cell-Profiling', 'data_subset', 'DMSO', 'B02_s2_w1